# Octave Convolution Tests

We can use this notebook to train and test our implementation of the OctConv module.

In [1]:
import torch
import torch.nn as nn

In [11]:
%load_ext autoreload
%autoreload 2

from modules import OctConv2d

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# Test a basic input to OctConv2d module

m = OctConv2d(3, 8, (3, 3))
input = torch.randn(128, 3, 27, 27)

output = m(input)
print(output.shape)